In [5]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from  torch.optim import Adam
import torch.nn.functional as ff


In [4]:
# !wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
# !unlink flowers
# !tar -xzf flower_data.tar.gz

In [6]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [7]:
train_transformers = transforms.Compose({
    transforms.Resize(224),
    transforms.RandomRotation(20),
    transforms.CenterCrop(220),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
})

test_transformers = transforms.Compose([
    transforms.CenterCrop(220),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load data
train_dataset = datasets.ImageFolder(train_dir, transform=train_transformers)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=12, shuffle=True)

test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transformers)
test_laoder = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=12, shuffle=True)


In [8]:
iamges, labels = next(iter(train_loader))
labels

tensor([27,  2,  9, 48, 21, 43, 58, 72, 18, 50, 40, 44])

In [10]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
cat_to_name, len(cat_to_name)

({'21': 'fire lily',
  '3': 'canterbury bells',
  '45': 'bolero deep blue',
  '1': 'pink primrose',
  '34': 'mexican aster',
  '27': 'prince of wales feathers',
  '7': 'moon orchid',
  '16': 'globe-flower',
  '25': 'grape hyacinth',
  '26': 'corn poppy',
  '79': 'toad lily',
  '39': 'siam tulip',
  '24': 'red ginger',
  '67': 'spring crocus',
  '35': 'alpine sea holly',
  '32': 'garden phlox',
  '10': 'globe thistle',
  '6': 'tiger lily',
  '93': 'ball moss',
  '33': 'love in the mist',
  '9': 'monkshood',
  '102': 'blackberry lily',
  '14': 'spear thistle',
  '19': 'balloon flower',
  '100': 'blanket flower',
  '13': 'king protea',
  '49': 'oxeye daisy',
  '15': 'yellow iris',
  '61': 'cautleya spicata',
  '31': 'carnation',
  '64': 'silverbush',
  '68': 'bearded iris',
  '63': 'black-eyed susan',
  '69': 'windflower',
  '62': 'japanese anemone',
  '20': 'giant white arum lily',
  '38': 'great masterwort',
  '4': 'sweet pea',
  '86': 'tree mallow',
  '101': 'trumpet creeper',
  '42': 

In [11]:
# TODO: Build and train your network
from collections import OrderedDict
model = models.VGG(features=True)

# freeze base layers
for param in model.parameters():
    param.requires_grad = False

# create classifier
classifier = nn.Sequential(OrderedDict({
    "fc1": nn.Linear(7, 250),
    "relu1": nn.ReLU(),
    "dropout": nn.Dropout(),
    "fc2": nn.Linear(250,90),
    "relu2": nn.ReLU(),
    "fc3": nn.Linear(90, 102),
    "output": nn.LogSoftmax(dim=1)
}))


model.classifier = classifier


In [14]:
from torch.optim import Adam

lr = 0.003
opochs = 20

optimizer = Adam(params=model.classifier.parameters(), lr=lr)
criterion = nn.NLLLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # device

model.to(device)

VGG(
  (avgpool): AdaptiveAvgPool2d(output_size=(7, 7))
  (classifier): Sequential(
    (fc1): Linear(in_features=7, out_features=250, bias=True)
    (relu1): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=250, out_features=90, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=90, out_features=102, bias=True)
    (output): LogSoftmax(dim=1)
  )
)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
